In [4]:
#Importing packages
import csv
from selenium import webdriver
import pandas as pd
import re
import time
import urllib
import ast
import json

In [22]:
def scrap(search_dict=None, debug=False, df=None):
    def log(message):
        global logs
        if not logs:
            logs = []
        if debug:
            print(message)
        logs.append(message)
    def get_settings():
        settings = {}
        with open('settings.csv') as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            line_count = 0
            for row in csv_reader:
                if line_count != 0:
                    settings[row[0]] = row[1]
                line_count += 1
            log(f'Processed {line_count} lines.')
        return settings
    
    def fb_login(driver,ctx):
        log('fb_login: %s, %s' % (driver,ctx))
        fb_email = ctx['fb_email']
        fb_password = ctx['fb_password']
        fb_login_button = ctx['fb_login_button']
        driver.get ("https://www.facebook.com")
        driver.find_element_by_id("email").send_keys(fb_email)
        driver.find_element_by_id("pass").send_keys(fb_password)
        driver.find_element_by_xpath(fb_login_button).click()
        return driver
    
    def scroll_to_bottom(driver):
        #Scroll to bottom infinity to load all posts

        SCROLL_PAUSE_TIME = 2

        # Get scroll height
        last_height = driver.execute_script("return document.body.scrollHeight")

        while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
        return driver 
    
    if search_dict is None:
        return
    log('Scrap: %s' % search_dict)
    global driver
    if driver is None:
        chrome_options = webdriver.ChromeOptions()
        prefs = {"profile.default_content_setting_values.notifications" : 2}
        chrome_options.add_experimental_option("prefs",prefs)
        driver = webdriver.Chrome('./../resources/chromedriver.exe',chrome_options=chrome_options)
    try:
        settings = get_settings()
        log('settings: %s' % settings)
        global login
        if login:
            login_ctx = {
                'fb_email': settings['fb_email'],
                'fb_password': settings['fb_password'],
                'fb_login_button': settings['fb_login_button']
            }
            driver = fb_login(driver,login_ctx)
            login = False
    #     search_dict = {
    #         "page_id": "373560576236",
    #         "page_name": "KEMENTERIAN KESIHATAN MALAYSIA",
    #         "filters": "eyJycF9jaHJvbm9fc29ydCI6IntcIm5hbWVcIjpcImNocm9ub3NvcnRcIixcImFyZ3NcIjpcIlwifSJ9",
    #         "search_term": "blood donation",
    #     }
        search_url = 'https://www.facebook.com/page/{}/search/?q={}&filters={}'.format(search_dict["page_id"],urllib.parse.quote(search_dict["search_term"]),search_dict["filters"])
        driver.execute_script("window.open('');")
        driver.switch_to.window(driver.window_handles[2])
        driver.get(search_url)
        scroll_to_bottom(driver)
        post_class = settings['post_class']
        regex_dict = {
            'date' : ast.literal_eval(settings['regex_date']),
            'title' : ast.literal_eval(settings['regex_title']),
            'reaction' : ast.literal_eval(settings['regex_reaction']),
            'comment' : ast.literal_eval(settings['regex_comment']),
            'share' : ast.literal_eval(settings['regex_share'])
        }
        log(repr(regex_dict))
        xpath = settings['xpath']
        vals = driver.find_elements_by_xpath(xpath.format(post_class))
        if df is None:
            df = pd.DataFrame()
        for ele in vals:
            log('ele: %s' % ele)
            info = {}
            for search_val,regex in regex_dict.items():
                match = re.search(regex,ele.text)
                if match:
                    info[search_val] = match.group(1)
                else:
                    info[search_val] = None
            info['raw'] = ele.text
            info.update(search_dict)
            log('\n-------\ninfo:\n%s' % info)
            if len(df) > 0:
                existing = df[(df["date"]==info["date"]) & (df["title"]==info["title"]) & (df["page_id"]==info["page_id"])]
                if len(existing) > 0:
                    log("existing:\n%s\n%s"% (info,existing))
                    continue
            df = df.append(info,ignore_index=True)
            log('df:\n%s' % df)
    except Exception as inst:
        log(inst)
#     driver.quit()
    driver.close()
    driver.switch_to.window(driver.window_handles[0])
    return df

In [56]:
page_done = []
df_done = pd.DataFrame()

In [62]:
#df = pd.read_csv("data.csv", index_col=0)
df = None
logs = None
driver = None
login = True
pages = pd.read_csv("pages.csv")
search_terms = pd.read_csv("search_terms.csv",header=None)[0].tolist()
date_filters = json.load(open('date_filters.json'))
for index, row in pages.iterrows():
    for search_term in search_terms:
        for (y,f) in date_filters.items():
            search_dict = {
                "page_id": str(row["Page ID"]),
                "page_name": row["Page Name"],
                "filters": f,
                "search_term": search_term,
            }
            check = search_dict['page_id'] + search_dict['filters'] + search_dict['search_term']
            if check in page_done:
                continue
            df_done = df_done.append(scrap(search_dict))
            page_done.append(check)
            time.sleep(5)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: DeprecationWarning: use options instead of chrome_options


In [86]:
df = df_done.drop(['level_0','index'], axis=1)
df['date'] = df_done['raw'].apply(lambda x: re.search("\n(\d{1,2}\s\w{3}\s\d{4})\n",x).group(1))
df.describe()

,comment,date,filters,page_id,page_name,raw,reaction,search_term,share,title
count,2169,3892,3892,3892,3892,3892,3029,3892,873,3726
unique,143,1442,5,17,17,3726,368,2,44,3221
top,1,31 Jul 2017,eyJycF9jcmVhdGlvbl90aW1lOjAiOiJ7XCJuYW1lXCI6XC...,402806916456377,Pusat Darah Negara Kementerian Kesihatan Malaysia,Kempen Derma Darah Bulan Sabit Merah SPT\nPage...,7,Derma Darah,1,"…""Like & Follow ""untuk info terkini mengenai d..."
freq,401,41,934,474,474,36,83,2431,240,43


In [87]:
df.to_csv('data.csv')
df

,comment,date,filters,page_id,page_name,raw,reaction,search_term,share,title
0,8,30 Jan 2015,eyJycF9jcmVhdGlvbl90aW1lOjAiOiJ7XCJuYW1lXCI6XC...,373560576236,KEMENTERIAN KESIHATAN MALAYSIA,KEMENTERIAN KESIHATAN MALAYSIA\nPage · 3.4M li...,364,Blood Donation,None,Schedule & Location of Blood Donation Campaign...
1,1,11 Dec 2015,eyJycF9jcmVhdGlvbl90aW1lOjAiOiJ7XCJuYW1lXCI6XC...,373560576236,KEMENTERIAN KESIHATAN MALAYSIA,KEMENTERIAN KESIHATAN MALAYSIA\nPage · 3.4M li...,391,Blood Donation,None,"""Donating blood is not a right. It's a privile..."
2,None,24 Jul 2015,eyJycF9jcmVhdGlvbl90aW1lOjAiOiJ7XCJuYW1lXCI6XC...,373560576236,KEMENTERIAN KESIHATAN MALAYSIA,KEMENTERIAN KESIHATAN MALAYSIA\nPage · 3.4M li...,None,Blood Donation,None,Did you know where your donated blood goes?
3,6,16 Nov 2015,eyJycF9jcmVhdGlvbl90aW1lOjAiOiJ7XCJuYW1lXCI6XC...,373560576236,KEMENTERIAN KESIHATAN MALAYSIA,KEMENTERIAN KESIHATAN MALAYSIA\nPage · 3.4M li...,142,Blood Donation,None,“You’re Somebody’s Type” Blood Donation Drive ...
4,None,3 Apr 2015,eyJycF9jcmVhdGlvbl90aW1lOjAiOiJ7XCJuYW1lXCI6XC...,373560576236,KEMENTERIAN KESIHATAN MALAYSIA,KEMENTERIAN KESIHATAN MALAYSIA\nPage · 3.4M li...,None,Blood Donation,None,…how to use mosquito fogging guns and donate b...
...,...,...,...,...,...,...,...,...,...,...
3887,None,15 Jul 2018,eyJycF9jcmVhdGlvbl90aW1lOjAiOiJ7XCJuYW1lXCI6XC...,162679250462156,Blood Donation Campaign Malaysia ????????,Blood Donation Campaign Malaysia 马来西亚捐血运动\nPag...,None,Derma Darah,None,Jadual Kempen Derma Darah 16 Julai 2018 (Isnin...
3888,1,2 Jul 2018,eyJycF9jcmVhdGlvbl90aW1lOjAiOiJ7XCJuYW1lXCI6XC...,162679250462156,Blood Donation Campaign Malaysia ????????,Blood Donation Campaign Malaysia 马来西亚捐血运动\nPag...,12,Derma Darah,None,SELANGOR JADUAL KEMPEN DERMA DARAH 02 JUL - 08...
3889,None,25 Jul 2018,eyJycF9jcmVhdGlvbl90aW1lOjAiOiJ7XCJuYW1lXCI6XC...,162679250462156,Blood Donation Campaign Malaysia ????????,Blood Donation Campaign Malaysia 马来西亚捐血运动\nPag...,7,Derma Darah,4,Jadual Kempen Derma Darah 26 Julai 2018 (Khami...
3890,None,19 Jul 2018,eyJycF9jcmVhdGlvbl90aW1lOjAiOiJ7XCJuYW1lXCI6XC...,162679250462156,Blood Donation Campaign Malaysia ????????,Blood Donation Campaign Malaysia 马来西亚捐血运动\nPag...,13,Derma Darah,1,Jadual Kempen Derma Darah 20 Julai 2018 (Jumaa...


In [88]:
for log in logs:
    print(log,'\n------')

Scrap: {'page_id': '587743868038803', 'page_name': 'Derma Darah Penang', 'filters': 'eyJycF9jcmVhdGlvbl90aW1lOjAiOiJ7XCJuYW1lXCI6XCJjcmVhdGlvbl90aW1lXCIsXCJhcmdzXCI6XCJ7XFxcInN0YXJ0X3llYXJcXFwiOlxcXCIyMDE3XFxcIixcXFwic3RhcnRfbW9udGhcXFwiOlxcXCIyMDE3LTFcXFwiLFxcXCJlbmRfeWVhclxcXCI6XFxcIjIwMTdcXFwiLFxcXCJlbmRfbW9udGhcXFwiOlxcXCIyMDE3LTEyXFxcIixcXFwic3RhcnRfZGF5XFxcIjpcXFwiMjAxNy0xLTFcXFwiLFxcXCJlbmRfZGF5XFxcIjpcXFwiMjAxNy0xMi0zMVxcXCJ9XCJ9In0', 'search_term': 'Derma Darah'} 
------
Processed 11 lines. 
------
settings: {'fb_email': 'chinyixiang@gmail.com', 'fb_password': 'chin0813', 'fb_login_button': "//form//*[(self::input or self::button) and (@value='Log In' or @value='Log Masuk' or text()='Log In' or text()='Log Masuk')]", 'post_class': 'rq0escxv l9j0dhe7 du4w35lb hybvsw6c ue3kfks5 pw54ja7n uo3d90p7 l82x9zwi ni8dbmo4 stjgntxs k4urcfbm sbcfpzgs', 'regex_date': '"\\n(\\d{1,2}\\w{3}\\s(\\s\\d{4})?)\\n"', 'regex_title': '"\\N{MIDDLE DOT}\\n\\s*\\N{MIDDLE DOT}\\s*(.*)\\n"', 'regex_reacti

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [2]:
import pandas as pd
df_test = pd.read_csv('data.csv',index_col=0)

In [3]:
df_test.sample(5)

,comment,date,filters,page_id,page_name,raw,reaction,search_term,share,title
1458,NaN,21 Jan 2019,eyJycF9jcmVhdGlvbl90aW1lOjAiOiJ7XCJuYW1lXCI6XC...,402806916456377,Pusat Darah Negara Kementerian Kesihatan Malaysia,Pusat Darah Negara Kementerian Kesihatan Malay...,19,Blood Donation,4.0,Lets donate blood on 23th & 24th January 2019 ...
2307,4.0,9 Sep 2015,eyJycF9jcmVhdGlvbl90aW1lOjAiOiJ7XCJuYW1lXCI6XC...,101247646587060,Kempen derma Darah Malaysia,Kempen derma Darah Malaysia\nPage · 12K like t...,6,Derma Darah,NaN,Penang...:)
3652,3.0,3 Jan 2015,eyJycF9jcmVhdGlvbl90aW1lOjAiOiJ7XCJuYW1lXCI6XC...,288697349159,Kempen Derma Darah Kepada Rakyat Malaysia,Kempen Derma Darah Kepada Rakyat Malaysia\nPag...,14,Derma Darah,NaN,…Darah Negara. Namun begitu kami bukan rasmi d...
230,235.0,30 Jun 2017,eyJycF9jcmVhdGlvbl90aW1lOjAiOiJ7XCJuYW1lXCI6XC...,373560576236,KEMENTERIAN KESIHATAN MALAYSIA,KEMENTERIAN KESIHATAN MALAYSIA\nPage · 3.4M li...,1.7K,Derma Darah,NaN,Bekalan Darah O kritikal di Hospital Melaka. #...
968,5.0,1 Apr 2018,eyJycF9jcmVhdGlvbl90aW1lOjAiOiJ7XCJuYW1lXCI6XC...,216075042059565,Derma Darah Ipoh,Derma Darah Ipoh\n11K like this · Blood bank\n...,57,Derma Darah,NaN,Moh komerr.. dtg derma darah dkt sini.. bkn ha...
